In [76]:
import pandas as pd
import re
from collections import Counter

In [111]:
train_df = pd.read_pickle("../../data/train_IOB_repl_compound.pkl")
test_df = pd.read_pickle("../../data/test_IOB_repl_compound.pkl")

In [72]:
def freq_headlines(df, n=None, _min=None):
    h_list = []
    for i, h in df.loc[:, 'h2':'h6'].iteritems():
        h_list += h.tolist()

    h_count = Counter(h_list)
    
    if n:
        return [h for h, _ in h_count.most_common(n + 1)]
    elif _min:
        return [h for h, count in h_count.most_common() if count >= _min]

def assign_cat_headline(df, headlines, col):
    '''
    Other headline         = 0
    Contains freq headline = 1
    No headline            = 2
    '''
    df[col] = 0

    df.loc[
        df.loc[:, 'h2':'h6'].fillna('NO_TITLE').apply(
            lambda x: x.str.contains('|'.join([re.escape(h) for h in headlines]))
        ).sum(1) > 0
        , col
    ] = 1

    df.loc[
        df.loc[:, 'h2':'h6'].isna().all(1)
        , col
    ] = 2
    
    return df

In [115]:
production_headlines = freq_headlines(train_df.loc[train_df.production_tag_seq.apply(lambda x: 'B' in x)], n=10)

train_df = assign_cat_headline(train_df, headlines=production_headlines, col='cat_production_headline')
test_df = assign_cat_headline(test_df, headlines=production_headlines, col='cat_production_headline')

In [116]:
raw_material_headlines = freq_headlines(train_df.loc[train_df.raw_material_tag_seq.apply(lambda x: 'B' in x)], n=10)

train_df = assign_cat_headline(train_df, headlines=raw_material_headlines, col='cat_raw_material_headline')
test_df = assign_cat_headline(test_df, headlines=raw_material_headlines, col='cat_raw_material_headline')

In [118]:
train_df.to_pickle("../../data/train_IOB_repl_compound.pkl")
test_df.to_pickle("../../data/test_IOB_repl_compound.pkl")